In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv 

train_image_names = []
train_mask_names = []

for root, _, files in os.walk('../input/ussimandsegm/abdominal_US/abdominal_US/AUS/images/train'):
    for i in files:
        img_path = os.path.join('../input/ussimandsegm/abdominal_US/abdominal_US/AUS/images/train', i)
        mask_path = os.path.join('../input/ussimandsegm/abdominal_US/abdominal_US/AUS/annotations/train', i)
        train_image_names.append(img_path)
        train_mask_names.append(mask_path)

for root, _, files in os.walk('../input/ussimandsegm/abdominal_US/abdominal_US/AUS/images/test'):
    for i in files:
        img_path = os.path.join('../input/ussimandsegm/abdominal_US/abdominal_US/AUS/images/test', i)
        mask_path = os.path.join('../input/ussimandsegm/abdominal_US/abdominal_US/AUS/annotations/test', i)
        train_image_names.append(img_path)
        train_mask_names.append(mask_path)

n=13
fig, axs = plt.subplots(1,2, figsize=[13,15])
image = cv.imread(train_image_names[n], 0)
#image = ImageOps.grayscale(image)
#image = np.array(image)
mask = cv.imread(train_mask_names[n])
#mask = ImageOps.grayscale(mask)
#mask = np.array(mask)
mask = np.where(mask==(10), (0), mask)
mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
axs[0].imshow(image)
axs[1].imshow(mask)
print(len(train_mask_names), len(train_image_names))
print(mask[0].shape)
print(np.unique(mask))

In [3]:
import pandas as pd

images = []
masks = []
img_height = 256
img_width = 256

for i in train_image_names:
    image = cv.imread(i)
    image = cv.resize(image, (img_height, img_width), interpolation = cv.INTER_AREA)
    image = image/255.0
    images.append(image)

for i in train_mask_names:
    mask = cv.imread(i)
    mask = np.where(mask==(10), (0), mask)
    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    mask = cv.resize(mask, (img_height, img_width), interpolation=cv.INTER_NEAREST)
    if 2 in np.unique(mask):
        print("Error") #sanity
    masks.append(mask)

images = np.array(images)
images = np.expand_dims(images, axis=3)
images = np.reshape(images, (-1, 256, 256, 3))
masks = np.array(masks)

In [4]:
print(np.unique(masks))
print(images.shape)

In [5]:
import tensorflow as tf

print(tf.__version__)

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Dropout, BatchNormalization, Activation, Lambda
from tensorflow.keras.layers import Cropping2D, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [6]:
def unet_model(no_classes, img_height, img_width):
    input = Input((img_height, img_width, 3), name='images')
    s = input

    c1 = Conv2D(16, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(s)
    c1 = BatchNormalization()(c1)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPool2D((2,2))(c1)

    c2 = Conv2D(32, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPool2D((2,2))(c2)

    c3 = Conv2D(64, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Dropout(0.1)(c3)
    c3 = Conv2D(64, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPool2D((2,2))(c3)    

    c4 = Conv2D(128, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Dropout(0.1)(c4)
    c4 = Conv2D(128, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPool2D((2,2))(c4)

    c5 = Conv2D(256, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Dropout(0.1)(c5)
    c5 = Conv2D(256, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c5)
    c5 = BatchNormalization()(c5)
    p5 = MaxPool2D((2,2))(c5)

    c6 = Conv2D(512, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(p5)
    c6 = BatchNormalization()(c6)
    c6 = Dropout(0.1)(c6)
    c6 = Conv2D(512, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c6)
    c6 = BatchNormalization()(c6)

    u7 = Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(c6)
    u7 = concatenate([u7, c5])
    c7 = Conv2D(256, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Dropout(0.1)(c7)
    c7 = Conv2D(256, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c7)
    c7 = BatchNormalization()(c7)

    u8 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c7)
    u8 = concatenate([u8, c4])
    c8 = Conv2D(128, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(128, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c8)
    c8 = BatchNormalization()(c8)

    u9 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c8)
    u9 = concatenate([u9, c3])
    c9 = Conv2D(64, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(64, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c9)
    c9 = BatchNormalization()(c9)

    u10 = Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c9)
    u10 = concatenate([u10, c2])
    c10 = Conv2D(32, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(u10)
    c10 = BatchNormalization()(c10)
    c10 = Dropout(0.1)(c10)
    c10 = Conv2D(32, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c10)
    c10 = BatchNormalization()(c10)

    u11 = Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(c10)
    u11 = concatenate([u11, c1])
    c11 = Conv2D(16, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(u11)
    c11 = BatchNormalization()(c11)
    c11 = Dropout(0.1)(c11)
    c11 = Conv2D(16, (3,3), activation='relu', kernel_initializer = 'he_normal',padding='same')(c11)
    c11 = BatchNormalization()(c11)

    output = Conv2D(no_classes, (1,1), activation="softmax", name='masks')(c11)

    model = Model(inputs=[input], outputs=[output])
    
    return model

In [7]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_masks, test_masks = train_test_split(images, masks, test_size=0.1)

In [8]:
plt.imshow(test_images[0])

In [9]:
print(train_images.shape, train_masks.shape)

In [22]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

def data_generator(images, masks, batch_size=8):
    while True:
        i=0
        while i<len(images)//batch_size:
            image_batch=np.zeros((batch_size, img_height, img_width, 3))
            mask_batch=np.zeros((batch_size, img_height, img_width))

            for j in range(batch_size):
                image, mask = images[i*j+j], masks[i*j+j]
                image_batch[j] = image
                mask_batch[j] = mask
                labelencoder = LabelEncoder()
                n, h, w = mask_batch.shape
                mask_batch_reshaped = mask_batch.reshape(-1, 1)
                mask_batch_reshaped_encoded = labelencoder.fit_transform(mask_batch_reshaped)
                mask_batch_encoded = mask_batch_reshaped_encoded.reshape(n, h, w)
                mask_batch_encoded = np.expand_dims(mask_batch_encoded, axis=3)
                mask_batch_cat = to_categorical(mask_batch_encoded, num_classes=9)
            i+=1
            yield {'images':image_batch}, {'masks':mask_batch_cat}

In [11]:
n, h, w = test_masks.shape
labelencoder = LabelEncoder()
test_masks_reshaped = test_masks.reshape(-1, 1)
test_masks_reshaped_encoded = labelencoder.fit_transform(test_masks_reshaped)
test_masks_encoded = test_masks_reshaped_encoded.reshape(n, h, w)
test_masks_encoded = np.expand_dims(test_masks_encoded, axis=3)

np.unique(test_masks_encoded)

In [12]:
def get_model():
    return unet_model(no_classes=9, img_height=img_height, img_width=img_width)

In [13]:
def jaccard_loss(y_true, y_pred, smooth=1):
    """
    Arguments:
        y_true : Matrix containing one-hot encoded class labels 
                 with the last axis being the number of classes.
        y_pred : Matrix with same dimensions as y_true.
        smooth : smoothing factor for loss function.
    """

    intersection = tf.reduce_sum( y_true * y_pred, axis=-1)
    union = tf.reduce_sum(y_true + y_pred, axis=-1) - intersection
    jac = (intersection + smooth) / (union + smooth)
    
    return (1 - jac) * smooth

In [14]:
model=get_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[jaccard_loss])
model.summary()

In [24]:
history = model.fit(data_generator(train_images, train_masks, 8),
                    steps_per_epoch = len(train_images)//8,
                    validation_data=data_generator(test_images, test_masks, 8),
                    validation_steps = len(test_images)//8,
                    epochs=50)#200 zhalele

In [31]:
n=5
fig, axs = plt.subplots(1,3, figsize=[13,15])
image = test_images[n]
image = cv.resize(image, (256,256), interpolation=cv.INTER_AREA)
#image = image/255.0
image_input = np.expand_dims(image, axis=0)
mask = test_masks[n]
'''mask = np.where(mask==(10), (0), mask)
mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
mask = cv.resize(mask, (256, 256), interpolation = cv.INTER_NEAREST)'''
prediction = model.predict(image_input)
#print(prediction.shape)
prediction_img = np.argmax(prediction[:,:,:,0:], axis=3)[0]
axs[0].imshow(image)
axs[1].imshow(mask, cmap='jet')
axs[2].imshow(prediction_img, cmap='jet')
#print(prediction_img)